In [92]:
# get books from goodreads metadata

import pandas as pd
import pickle
metadata_goodreads = pickle.load(open( "pickled_metadata/metadata_goodreads.p", "rb" ))
books = list(set(metadata_goodreads['clean_title']))

In [93]:
books = [b for b in books if len(b.split())<3]

In [94]:
books;

In [95]:
def prepro_jsonl_dataset(ds,augument_chance=0,book_list_to_pick=None,remove_negatives=True):
    new_ds = []
    
    for d in ds:

        if d['entities']:

            if(np.random.rand()>1-augument_chance):

                d_ = d.copy()
                book = d_['entities'][0]['text']
                book_to_change = np.random.choice(book_list_to_pick,1)[0]
                temp = d_['text'].replace(book,book_to_change)
                d_['text'] = temp


                b = temp.find(book_to_change)
                e = b + len(book_to_change)

                #dict(start=5,end=14,text="Tal Perry",label="Person")
                entities = {
                    'start' : b,
                    'end' : e,
                    'text' : book_to_change,
                    'label' : "OBRA",
                    'tag' : "OBRA"
                }

                d_['meta']['book'] = book_to_change

                d_['entities'] = [entities]
                
                # add original and augumented
                new_ds.append(d_)
            new_ds.append(d)   

        else:
            if(not remove_negatives):
                print('opa')
                new_ds.append(d) 
        

    return new_ds

In [96]:
import json

def load_jsonl_dataset(path = None):
    with open(path,'r') as f:
        return list(map(lambda x : json.loads(x),f.readlines()))

ds_odissey  = load_jsonl_dataset ('annotated/odyssey_annotaded.jsonl') 
ds_calibre = load_jsonl_dataset ('annotated/dataset_annotated-fixed-done.jsonl') 
ds_crimeandpun =  load_jsonl_dataset ('annotated/crimeandpun.jsonl') 
ds_republic =  load_jsonl_dataset ('annotated/therepublic.jsonl') 
ds_goodreads = load_jsonl_dataset ('annotated/smaller_dataset_goodreads(ANNOTATED).jsonl') 

In [97]:
import numpy as np
np.random.seed(1337)


In [98]:
ds_calibre = prepro_jsonl_dataset(ds_calibre,augument_chance=0.9,book_list_to_pick=books,remove_negatives=True)
ds_odissey = prepro_jsonl_dataset(ds_odissey,augument_chance=.9,book_list_to_pick=books,remove_negatives=True)
ds_crimeandpun = prepro_jsonl_dataset(ds_crimeandpun,augument_chance=.9,book_list_to_pick=books,remove_negatives=True)
ds_republic = prepro_jsonl_dataset(ds_republic,augument_chance=.9,book_list_to_pick=books,remove_negatives=True)

#### Goodreads dataset needs preproc

In [99]:
for d in ds_goodreads:
    book = d['meta']['book']
    
    temp = d['text'].replace('[[BOOK]]',book)
    d['text'] = temp
    b = temp.find(book)
    e = b + len(book)

    #dict(start=5,end=14,text="Tal Perry",label="Person")
    entities = {
        'start' : b,
        'end' : e,
        'text' : book,
        'label' : "OBRA",
        'tag' : "OBRA"
    }
    if(d['labels']==['negative']):
        d['entities'] = []
    else:
        d['entities'] = [entities]

In [100]:
dsi = []
list_ds = [ds_calibre,ds_odissey,ds_crimeandpun,ds_republic]
list_ds = [ds_calibre]
for d in list_ds:

    dsi.extend(d)
print(len(dsi))

1153


In [101]:
import random

random.shuffle(dsi)

dataset_train = dsi[:-50]
dataset_test = dsi[-50:]

In [102]:
#with open('ner_datasets/dataset_annotated_ner_calibre.jsonl','w') as f:
#    for js in ds: 
#        f.write(json.dumps(js) + '\n')
#    f.close()

### convert ENTITIES to Hugginface Format (tokens)

In [103]:
from transformers import RobertaTokenizerFast, RobertaModel
import torch




In [104]:
from typing_extensions import TypedDict
from typing import List,Any
IntList = List[int] # A list of token_ids
IntListList = List[IntList] # A List of List of token_ids, e.g. a Batch

def align_tokens_and_annotations_bilou(tokenized, annotations):
    tokens = tokenized.tokens
    aligned_labels = ["O"] * len(
        tokens
    )  # Make a list to store our labels the same length as our tokens
    for anno in annotations:
        annotation_token_ix_set = (
            set()
        )  # A set that stores the token indices of the annotation
        for char_ix in range(anno["start"], anno["end"]):

            token_ix = tokenized.char_to_token(char_ix)
            if token_ix is not None:
                annotation_token_ix_set.add(token_ix)
        if len(annotation_token_ix_set) == 1:
            # If there is only one token
            token_ix = annotation_token_ix_set.pop()
            prefix = (
                "U"  # This annotation spans one token so is prefixed with U for unique
            )
            aligned_labels[token_ix] = f"{prefix}-{anno['label']}"

        else:

            last_token_in_anno_ix = len(annotation_token_ix_set) - 1
            for num, token_ix in enumerate(sorted(annotation_token_ix_set)):
                if num == 0:
                    prefix = "B"
                elif num == last_token_in_anno_ix:
                    prefix = "L"  # Its the last token
                else:
                    prefix = "I"  # We're inside of a multi token annotation
                aligned_labels[token_ix] = f"{prefix}-{anno['label']}"
    return aligned_labels


import itertools


class LabelSet:
    def __init__(self, labels: List[str]):
        self.labels_to_id = {}
        self.ids_to_label = {}
        self.labels_to_id["O"] = 0
        self.ids_to_label[0] = "O"
        num = 0  # in case there are no labels
        # Writing BILU will give us incremntal ids for the labels
        for _num, (label, s) in enumerate(itertools.product(labels, "BILU")):
            num = _num + 1  # skip 0
            l = f"{s}-{label}"
            self.labels_to_id[l] = num
            self.ids_to_label[num] = l
        # Add the OUTSIDE label - no label for the token

    def get_aligned_label_ids_from_annotations(self, tokenized_text, annotations):
        raw_labels = align_tokens_and_annotations_bilou(tokenized_text, annotations)    
        return list(map(self.labels_to_id.get, raw_labels))
from dataclasses import dataclass
from torch.utils.data import Dataset
from transformers import PreTrainedTokenizerFast

@dataclass
class TrainingExample:
    input_ids: IntList
    attention_masks: IntList
    labels: IntList


class TraingDataset(Dataset):
    def __init__(
        self,
        data: Any,
        label_set: LabelSet,
        tokenizer: PreTrainedTokenizerFast,
        tokens_per_batch=32,
        window_stride=None,
    ):
        self.label_set = label_set
        if window_stride is None:
            self.window_stride = tokens_per_batch
        self.tokenizer = tokenizer
        for example in data:
            # changes tag key to label
            for a in example["entities"]:
                a["label"] = a["tag"]
                pass
        self.texts = []
        self.annotations = []

        for example in data:
            self.texts.append(example["text"])
            self.annotations.append(example["entities"])
        ###TOKENIZE All THE DATA
        tokenized_batch = self.tokenizer(self.texts, add_special_tokens=False)
        ###ALIGN LABELS ONE EXAMPLE AT A TIME
        aligned_labels = []
        for ix in range(len(tokenized_batch.encodings)):
            encoding = tokenized_batch.encodings[ix]
            raw_annotations = self.annotations[ix]            
            aligned = label_set.get_aligned_label_ids_from_annotations(
                encoding, raw_annotations
            )
            aligned_labels.append(aligned)
        ###END OF LABEL ALIGNMENT

        ###MAKE A LIST OF TRAINING EXAMPLES. (This is where we add padding)
        self.training_examples: List[TrainingExample] = []
        empty_label_id = "O"
        for encoding, label in zip(tokenized_batch.encodings, aligned_labels):
            length = len(label)  # How long is this sequence
            for start in range(0, length, self.window_stride):

                end = min(start + tokens_per_batch, length)

                # How much padding do we need ?
                padding_to_add = max(0, tokens_per_batch - end + start)
                self.training_examples.append(
                    TrainingExample(
                        # Record the tokens
                        input_ids=encoding.ids[start:end]  # The ids of the tokens
                        + [self.tokenizer.pad_token_id]
                        * padding_to_add,  # padding if needed
                        labels=(
                            label[start:end]
                            + [-100] * padding_to_add  # padding if needed
                        ),  # -100 is a special token for padding of labels,
                        attention_masks=(
                            encoding.attention_mask[start:end]
                            + [0]
                            * padding_to_add  # 0'd attenetion masks where we added padding
                        ),
                    )
                )

    def __len__(self):
        return len(self.training_examples)

    def __getitem__(self, idx) -> TrainingExample:

        return self.training_examples[idx]

In [105]:
class TraingingBatch:
    def __getitem__(self, item):
        return getattr(self, item)

    def __init__(self, examples: List[TrainingExample]):
        self.input_ids: torch.Tensor
        self.attention_masks: torch.Tensor
        self.labels: torch.Tensor
        input_ids: IntListList = []
        masks: IntListList = []
        labels: IntListList = []
        for ex in examples:
            input_ids.append(ex.input_ids)
            masks.append(ex.attention_masks)
            labels.append(ex.labels)
        self.input_ids = torch.LongTensor(input_ids)
        self.attention_masks = torch.LongTensor(masks)
        self.labels = torch.LongTensor(labels)

In [106]:
label_set = LabelSet(labels=["OBRA"])
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

ds = TraingDataset(
    data=dataset_train, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=40
)

Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors


In [107]:
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AdamW
from torch.utils.data.dataloader import DataLoader
from transformers import RobertaForTokenClassification


model = RobertaForTokenClassification.from_pretrained(
    "roberta-base", num_labels=len(ds.label_set.ids_to_label.values())
)

optimizer = AdamW(model.parameters(), lr=5e-5)
nb_grad_acc_steps = 2
warmup_steps = 1
num_epochs = 5

dataloader = DataLoader(
    ds,
    collate_fn=TraingingBatch,
    batch_size=4,
    shuffle=True,
)



nb_training_steps = len(dataloader)//nb_grad_acc_steps

scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,num_training_steps=nb_training_steps)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [108]:
model.cuda();


In [109]:
import numpy as np
from sklearn.metrics import classification_report

def check_accuracy_model(model,dataset_raw):

    outs = []
    for d in dataset_raw:

            y_pred = ''
            if len(d['entities'])>0:
            
                y_true = d['entities'][0]['text']
            else:
                y_true = ''
                
            inputs = tokenizer(d['text'], return_tensors="pt",truncation=True,padding=True).to(device)

            logits = model(**inputs)

            idx = torch.nonzero(torch.argmax(logits[0][0],axis=1))

            if (len(idx)):
                ids_labeled = inputs.input_ids[0][idx]
                tokens = tokenizer.convert_ids_to_tokens(ids_labeled)
                y_pred = tokenizer.convert_tokens_to_string(tokens).strip()

                success = y_pred.find(y_true)!=-1
                outs.append(success)
            else:
                pass
                
            outs.append(y_pred==y_true)

                

    return np.mean(outs)


In [110]:
from tqdm import tqdm





for epoch in range(num_epochs):
    iterator = dataloader.__iter__()

    loop = tqdm(range(int(nb_training_steps)))
    optimizer.zero_grad()
    
    for i_train in loop:
        
        batch = next(iterator)
            
        device = 'cuda'
        input_ids = batch.input_ids.to(device)
        attention_masks = batch.attention_masks.to(device)
        labels = batch.labels.to(device)

        loss, logits = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            labels=labels,
            return_dict=False
        )
        
        loss = loss/nb_grad_acc_steps
        loss.backward()  
        if (i_train+1) % nb_grad_acc_steps == 0:      

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()   
            
        loop.set_description(f'Epoch={epoch}/{num_epochs}')
        loop.set_postfix(loss=loss.item())
        
    #gd_val = check_accuracy_model(model,ds_goodreads)
    val_val = check_accuracy_model(model,dataset_test)

    print(f'Epoch val loss Goodreads:{gd_val} Val Set:{val_val}')
    

Epoch=1/5:   1%|          | 2/395 [00:00<00:23, 16.65it/s, loss=0.0154]

Epoch val loss Goodreads:0.5887096774193549 Val Set:0.71


Epoch=2/5:   1%|          | 2/395 [00:00<00:24, 15.87it/s, loss=0.0198] 

Epoch val loss Goodreads:0.5887096774193549 Val Set:0.7551020408163265


Epoch=3/5:   1%|          | 2/395 [00:00<00:24, 16.07it/s, loss=0.00465]

Epoch val loss Goodreads:0.5887096774193549 Val Set:0.7551020408163265


Epoch=4/5:   1%|          | 2/395 [00:00<00:23, 16.74it/s, loss=0.0149]

Epoch val loss Goodreads:0.5887096774193549 Val Set:0.7551020408163265


Epoch=4/5: 100%|██████████| 395/395 [00:23<00:00, 16.77it/s, loss=0.0129]  


Epoch val loss Goodreads:0.5887096774193549 Val Set:0.7551020408163265


torch.device

In [119]:
import os
model.save_pretrained(f"fine-tuned-model-ner-better-data-3")

In [ ]:
dataset_test

In [84]:
ds.label_set.ids_to_label.values()

dict_values(['O', 'B-OBRA', 'I-OBRA', 'L-OBRA', 'U-OBRA'])

## TEST MODEL

In [1]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification
import torch

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = RobertaForTokenClassification.from_pretrained("fine-tuned-model-ner-better-data-3/")

/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lira/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
from utils import check_if_citation_ner
model.cpu();

In [3]:
check_if_citation_ner(model,tokenizer,'Reading The Prince and The Republic has changed my life')

'The Prince The Republic'

In [4]:
check_if_citation_ner(model,tokenizer,'The prince reads Marcus Aurelius Meditations to relax')

'Meditations'

In [5]:
check_if_citation_ner(model,tokenizer,"When Odysseus returns to Ithaca in Book 13 of The Odyssey, Athena disguises him as an old beggar")

'The Odyssey'

In [6]:
check_if_citation_ner(model,tokenizer,"But these more outré materials combine to form what is finally a thematic second tier. The foreground of Infinite Jest features three basic plot systems. At the center of one is Hal Incandenza, an adolescent tennis star attending Enfield Tennis Academy (ETA), which his family founded,")

'Infinite Jest'

In [7]:
check_if_citation_ner(model,tokenizer,"the commencement of war a herald might be called upon to recite the causes of the conflict; in effect, to provide the motivation. In Shakespeare's Henry IV")

'Henry IV'

In [8]:
check_if_citation_ner(model,tokenizer,"is impersonal in the Meditations agrees closely with Epictetus. Marcus Aurelius is doubtful about immortality, but says, as a Christian might: 'Since it is possible th")

'Meditations'

In [9]:
check_if_citation_ner(model,tokenizer,"8 Melville’s Moby-Dick, for instance.")

'Moby-Dick'

In [10]:
import pickle
metadata_calibre = pickle.load(open( "pickled_metadata/calibre_metadata.p", "rb" ))
metadata_calibre.query('clean_title=="Story of Philosophy"').index

Int64Index([24], dtype='int64', name='id')

In [18]:
nietz = []
with open(f'books_raw/{186}.txt', 'r') as book_text:
    nietz.append(book_text.readlines())
    

In [38]:
for line in nietz[0]:
    if line.find('The New Republic')!=-1:
        sentence = line
        #print(check_if_citation_ner(model,tokenizer,line))

In [46]:
sentence = sentence+sentence

In [47]:
inputs = tokenizer(sentence, return_tensors="pt",truncation=True,padding=True)
inputs.input_ids[0].shape

torch.Size([512])

In [35]:
check_if_citation_ner(model,tokenizer,sentence)

''

#### Predicting on a Batch directly 

In [32]:
def check_if_citation_ner_multiline(model,tokenizer,sentences):

    inputs = tokenizer(sentences, return_tensors="pt",truncation=True,padding=True)

    logits = model(**inputs)
    idx = torch.nonzero(torch.argmax(logits[0][0],axis=1))



    num_sents = logits[0].shape[0]
    idx_sentence = torch.nonzero(torch.argmax(logits[0],axis=2)).numpy()

    dict_tokens_idx = {num:[] for num in range(num_sents)}

    for row,idx in idx_sentence:
        dict_tokens_idx[row].append(idx)
    books_found = []
    for row,ids in dict_tokens_idx.items():
        ids_labeled = inputs.input_ids[row][ids]
        tokens = tokenizer.convert_ids_to_tokens(ids_labeled)
        books_found.append(tokenizer.convert_tokens_to_string(tokens).strip())
    
    return books_found

### testing model

In [33]:
import json
with open('to_annotate/output_ner_multi.jsonl','r') as f:
    
    ds_goodreads = list(map(lambda x : json.loads(x),f.readlines()))

In [34]:
ds_goodreads[70]

{'text': "Art thou there, truepenny? -- Interesting only to the parish clerk. I mean, we have the plays. I mean when we read the poetry of King Lear what is it to us how the poet lived? As for living, our servants can do that for us, Villiers de l'Isle has said.",
 'labels': ['positive'],
 'detected_ner': '-1',
 'meta': {'book': 'King Lear'}}

In [35]:
sentence = "The manuscript of the first edition of Moby Dick was submitted in December 2009 and finalized in January 2010, after a version of the financial reform bill passed The Republic but before formal debate began in The Republic."

In [36]:
%%time
for _ in range(4):
    check_if_citation_ner(model,tokenizer,sentence)
    

CPU times: user 796 ms, sys: 2.65 ms, total: 799 ms
Wall time: 136 ms


In [37]:
%%time
check_if_citation_ner_multiline(model,tokenizer,[sentence]*4)

CPU times: user 543 ms, sys: 28 ms, total: 571 ms
Wall time: 100 ms


['Moby Dick The The',
 'Moby Dick The The',
 'Moby Dick The The',
 'Moby Dick The The']

In [38]:
check_if_citation_ner(model,tokenizer,ds_goodreads[70]['text'])

'King Lear'

In [45]:
use_citation_model = True
create_dataset = False

from utils import find_any_book_calibre,search_book_pubdate_by_alias

lookup_func = find_any_book_calibre

using_goodreads=False

from nltk.corpus import stopwords


def process_book(tup,model,tokenizer):
             
                        return (current_book_name,edges_to_add,pieces_dataset)

In [41]:
idx_book = 24

books_memory = []
with open(f'books_raw/{idx_book}.txt', 'r') as book_text:
    books_memory.append(book_text.readlines())
    
tup = metadata_calibre.index[idx_book],metadata_calibre['aliases'][idx_book],metadata_calibre['clean_title'][idx_book],books_memory[0]


In [39]:
%%time
process_book(tup,model,tokenizer)

NameError: name 'process_book' is not defined

In [43]:
from utils import BookProcesserFactory

bkp = BookProcesserFactory(create_dataset = False,
                    verbose = False,
                    use_citation_model =True,
                    metadata_to_use = 'calibre'       
                )

book_processer = bkp.GetProcessFunction()


In [44]:
%%time
book_processer(tup,model,tokenizer)

13
CPU times: user 13.8 s, sys: 0 ns, total: 13.8 s
Wall time: 2.89 s


('Story of Philosophy',
 [('Story of Philosophy', 'The Republic'),
  ('Story of Philosophy', 'The Republic'),
  ('Story of Philosophy', 'The Republic'),
  ('Story of Philosophy', 'Crito')],
 [])

In [26]:
books_memory[0][i:i+3]

NameError: name 'i' is not defined

In [ ]:
words = 'The Birth of Traged Untimely Meditations All Too Human Assorted Opin The Wand The Dawn The Gay Science Ecce Homo'

In [ ]:
from nltk.corpus import stopwords


In [ ]:
found = ['The Birth of Tragedy', 'Thus Spoke Zarathustra', 'Ecce Homo']

In [ ]:
for w in found:
    filtered_regex = [word for word in w.split() if not word.lower() in stopwords.words()]
    for w2 in filtered_regex:
        if words.find(w2)!=-1:
            print(w2)
            break

In [ ]:
filtered_regex